In [ ]:
# Импортируем нужные фунции из файлика db_func.py + создадим экземпляр SQLITE таблички с именем messages.db - в ней и будем хранить всю информацию
from db_func import conn, create_table_with_dialogs, store_dialog, create_table, store_message, create_table_with_users, store_user

In [ ]:
# Тут обозначим и запустим наш клиент - при запуске этой ячейки нужно будет ввести номер телефона, 
# ввести код подтверждения которое прийдет в ваш телеграм канал - с этого момента вы будете авторизованы 
# и сможете скачивать сообщения из тех каналов, на которые подписаны
API_ID = 12125354
API_HASH = '32b18b9170c167da78af810695ad7404'

from telethon import TelegramClient
client = TelegramClient('crawler_session', API_ID, API_HASH)

await client.start()

In [ ]:
# Создадим и заполним таблицу со всеми нашими диалогами
create_table_with_dialogs()
async for dialog in client.iter_dialogs():
    try:
        store_dialog(dialog.id,dialog.name)
    except:
        pass

Подключиться к файлу с БД SQLITE можно любым доступным средством - включая DBEAVER. Посмотрим id интересующих нас диалогов и поместим их в список  dialog_list

In [ ]:
dialog_list = []

In [ ]:
# Наполним табличку с пользователями инфой из интересующих нас диалогов - 
# Важно -  не из всех диалогов можно получить список пользователей(может быть ограничено админом канала)
create_table_with_users()
for dialog in dialog_list:
    try:
        part = await client.get_participants(dialog)
        print(dialog,'Получен список пользователей')
    except:
        part = None
        print(f'Для канала {dialog} нельзя запросить пользователей')
    try:
        for p in part:
            try:
                store_user(p.id,p.username,p.phone,p.first_name,p.last_name)
            except:
                pass
    except:
        pass

In [ ]:
# Выгрузим все текствые сообщения из интересующих нас каналов 
# Важно - выгрузка идет с последнего сообщения к первому - по одному (поэтому важно сделать первую выгрузку до конца) 
# При повтороном запуске скрипта таблицы с сообщениями будут только дополнены новыми сообщениями начиная с последнего полученного
for chanel in dialog_list:
        messages = await client.get_messages(chanel)
        print('Всего сообщений в канале',messages.total)
        i = 0
        create_table(chanel)
        c = conn.cursor()
        last_msg = c.execute(f'''SELECT MAX(message_id) FROM "{chanel}"''').fetchone()[0]
        conn.commit()
        async for message in client.iter_messages(chanel):
                if last_msg == None or message.id > last_msg:
                        if message.from_id:
                                author = message.from_id.user_id
                        elif message.peer_id:
                                try:
                                        author = message.peer_id.channel_id
                                except:
                                        author = message.peer_id.user_id
                        else:
                                author = 'unc'
                        store_message(chanel,message.id,author,message.text)
                else:
                        print('Все сообщения загружены')
                        break
                i = i+1
                if i % 1000 == 0:
                        print(f'Загружено {i} сообщений')